In [15]:
import os
import cv2
import numpy as np
from skimage.feature import local_binary_pattern
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelEncoder

In [16]:
# Function to load images and labels from a dataset folder
def load_dataset(dataset_path):
    images = []
    labels = []
    label_names = os.listdir(dataset_path)
    for label in label_names:
        label_path = os.path.join(dataset_path, label)
        for image_name in os.listdir(label_path):
            image_path = os.path.join(label_path, image_name)
            image = cv2.imread(image_path)
            images.append(image)
            labels.append(label)
    return images, labels

# Load dataset
dataset_path = 'fish_dt'  # Replace with your actual dataset path
images, labels = load_dataset(dataset_path)

In [17]:
# Convert images to grayscale
gray_images = [cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) for image in images]

In [18]:
# Apply LBP
def apply_lbp(image, num_points=24, radius=8):
    lbp = local_binary_pattern(image, num_points, radius, method='uniform')
    return lbp

In [19]:
# Apply LDP
def apply_ldp(image):
    gx = cv2.Sobel(image, cv2.CV_32F, 1, 0, ksize=3)
    gy = cv2.Sobel(image, cv2.CV_32F, 0, 1, ksize=3)
    magnitude, angle = cv2.cartToPolar(gx, gy, angleInDegrees=True)
    angle_bins = np.int32(8 * angle / 360)  # Quantize the angle to 8 bins
    return angle_bins

In [20]:
# Compute histogram
def compute_histogram(image, bins=16):
    hist, _ = np.histogram(image.ravel(), bins=bins, range=(0, bins))
    hist = hist.astype("float")
    hist /= (hist.sum() + 1e-6)  # Normalize the histogram
    return hist

# Convert labels to numerical values
label_encoder = LabelEncoder()
numeric_labels = label_encoder.fit_transform(labels)

In [21]:
# Function to train and evaluate classifiers
def train_and_evaluate_model(X_train, X_test, y_train, y_test, classifier):
    clf = classifier
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    return accuracy, precision, recall, f1

In [27]:
# Evaluate RGB channel features with different histogram bin sizes
def extract_color(image, bins):
    # Convert the image from RGB to YCbCr color space
    ycbcr_image = cv2.cvtColor(image, cv2.COLOR_RGB2YCrCb)
    channels = cv2.split(ycbcr_image)
    features = []
    for channel in channels:
        lbp_image = apply_lbp(channel)
        lbp_hist_features = compute_histogram(lbp_image, bins)
        ldp_image = apply_ldp(channel)
        ldp_hist_features = compute_histogram(ldp_image, bins)
        features.extend(np.hstack((lbp_hist_features, ldp_hist_features)))
    return features
bin_sizes = [8, 16, 32, 64, 128, 256]
results = []

for bins in bin_sizes:
    rgb_features_list = [extract_color(image, bins) for image in images]
    X_train, X_test, y_train, y_test = train_test_split(rgb_features_list, numeric_labels, test_size=0.2, random_state=42)
    accuracy, precision, recall, f1 = train_and_evaluate_model(X_train, X_test, y_train, y_test, AdaBoostClassifier(n_estimators=100, random_state=42))
    results.append((bins, accuracy, precision, recall, f1))


C:\Users\KIIT\AppData\Roaming\Python\Python312\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
C:\Users\KIIT\AppData\Roaming\Python\Python312\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
C:\Users\KIIT\AppData\Roaming\Python\Python312\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
C:\Users\KIIT\AppData\Roaming\Python\Python312\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the S

In [28]:
# Print results in a tabular format
print(f"{'Bins':<10}{'Accuracy':<10}{'Precision':<10}{'Recall':<10}{'F1 Score':<10}")
for bins, accuracy, precision, recall, f1 in results:
    print(f"{bins:<10}{accuracy:<10.4f}{precision:<10.4f}{recall:<10.4f}{f1:<10.4f}")

Bins      Accuracy  Precision Recall    F1 Score  
8         0.8657    0.8619    0.8657    0.8637    
16        0.8743    0.8673    0.8743    0.8704    
32        0.8629    0.8655    0.8629    0.8641    
64        0.8629    0.8655    0.8629    0.8641    
128       0.8629    0.8655    0.8629    0.8641    
256       0.8629    0.8655    0.8629    0.8641    
